<a href="https://colab.research.google.com/github/NinelK/SA_DS_tutorial/blob/main/DS_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial:** Dynamical systems in neuroscience

## Introduction

Dynamical systems are systems that **evolve** in time. These can be physical systems, economic systems, neurons, neural networks or the whole brain. No matter what the system is, the universal dynamical system framework can help us understand its time-dependent behavior.

When modelling dynamics, we can represent time in two different ways: *discrete* or *continuous*. In both cases, we can write down the evolution of the system as a function of its previous `state' $x$ and external inputs that the system receives $u$:

Discrete time systems               |           Continuous time systems
------------------------------------|-------------------------------------
 $$x_{t+1} = F_d(x_t,u_t)$$           |    $$\frac{dx(t)}{dt} = F_c(x(t),u(t))$$

Here, the state $x$ can, for instance, correspont to neural firing. In discrete case, $x_t$ would be the number of spikes emmited within one time bin, while $x(t)$ could be a function which is only non-zero at spike times.
External inputs $u$ in this case would correspond to the inputs from other neurons.

The key part of the dynamical systems framework is the **evolution operator** $F$. Whether continuous $F_c$ or discrete $F_d$, it can tell us a lot about the system:
> The power of the dynamical systems approach to neuroscience, as well as to many other sciences, is that we can tell something, or many things, about a system without knowing all the details that govern the system evolution. We do not even use equations to do that! Some may even wonder why we call it a mathematical theory.        *Eugene Izhikevich [1]*

Therefore, the main goal of the tutorial is to learn how to tell something about the dynamical system knowing $F$.

# Part 1: Autonomous linear dynamical systems

Sometimes, we can assume that the system does not receive any external inputs and evolves in time on its own ( $u(t)=0$ ). Such dynamical systems are called **autonomous**. Such systems are common in physics (e.g. swinging pendulum), but also surprisingly applicable to some biological neural networks, as we will see in the last part of the tutorial. Lack of external inputs greatly simplifies the analysis of the evolution $F$, so let us assume no external inputs for now.

Let us start 

## **Exercise 1**: relationship between continuous time and discrete time linear dynamical systems

Suppose we know the state $x(0)$ of a continuous-time dynamical system and we want to know what happens next. To figure this out, we need to integrate a continuous-time dynamical system equation over time.

## Part 1 objectives

1. Learn what linear systems can and can not do
2. Learn what fixed points are
3. Read eignespectra

# Part 2: Nonlinear dynamical systems

## Fitzhugh-Nagumo model of a spiking neuron

Hodgkin Huxley -> simplify

Nullclines

Phase portrait

# Part 3: Latent dynamics models

In recent years, progress in recording technologies enabled recordings of 100s to 1000s of neurons simultaneously.

At the same time, it was shown that neural population activity often has a low-dimensional structure: a low number of latent dynamical factors can explain a large fraction of neural variability. This finding is called a 'manifold hypothesis', and was proposed in [REF].



WRITE INTRO

Define Latent dynamics and Emission model

## Recurrent neural networks (RNNs)

Let us first consider the simplest, yet most important example of a non-linear discrete-time latent dynamical system: recurrent neural network. It is a go-to tool for modelling sequences in deep learning.

Latent dynamics: $$h_{t} = \sigma (V h_{t-1} + U x_t + b_h)$$
Emission model: $$o_t = W h_t + b_o $$


![RNN scheme (Wiki)](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Recurrent_neural_network_unfold.svg/2560px-Recurrent_neural_network_unfold.svg.png)


## **Exercise 3.1** Implement VanillaRNN



In [ ]:
class VanillaRNN(nn.Module):
  def __init__(self, output_size, hidden_size, vocab_size, embed_size):
    super(VanillaRNN, self).__init__()
      ####################################################################
    # Fill in missing code below (...),
    # then remove or comment the line below to test your function
    raise NotImplementedError("Vanilla RNN")
    ####################################################################
    self.hidden_size = ...
    self.neuron_embeddings = ...
    self.rnn = ...
    self.fc = ...

    self.hidden_size = hidden_size
    self.word_embeddings = nn.Embedding(vocab_size, embed_size)
    self.rnn = nn.RNN(embed_size, hidden_size, num_layers=2)
    self.fc = nn.Linear(2*hidden_size, output_size)

  def forward(self, inputs):
    input = self.neuron_embeddings(inputs)
    input = input.permute(1, 0, 2)
    h_0 =  Variable(torch.zeros(2, input.size()[1], self.hidden_size)
    output, h_n = self.rnn(input, h_0)
    h_n = h_n.permute(1, 0, 2)
    h_n = h_n.contiguous().view(h_n.size()[0], h_n.size()[1]*h_n.size()[2])
    logits = self.fc(h_n)

    return logits

## Uncomment to test
# sampleRNN = VanillaRNN(10, 50, 1000, 300)
# print(sampleRNN)

In [ ]:
class VanillaRNN(nn.Module):
  def __init__(self, output_size, hidden_size, vocab_size, embed_size):
    super(VanillaRNN, self).__init__()
      ####################################################################
    # Fill in missing code below (...),
    # then remove or comment the line below to test your function
    raise NotImplementedError("Vanilla RNN")
    ####################################################################
    self.hidden_size = ...
    self.neuron_embeddings = ...
    self.rnn = ...
    self.fc = ...

    self.hidden_size = hidden_size
    self.word_embeddings = nn.Embedding(vocab_size, embed_size)
    self.rnn = nn.RNN(embed_size, hidden_size, num_layers=2)
    self.fc = nn.Linear(2*hidden_size, output_size)

  def forward(self, inputs):
    input = self.neuron_embeddings(inputs)
    input = input.permute(1, 0, 2)
    h_0 =  Variable(torch.zeros(2, input.size()[1], self.hidden_size)
    output, h_n = self.rnn(input, h_0)
    h_n = h_n.permute(1, 0, 2)
    h_n = h_n.contiguous().view(h_n.size()[0], h_n.size()[1]*h_n.size()[2])
    logits = self.fc(h_n)

    return logits

## Uncomment to test
# sampleRNN = VanillaRNN(10, 50, 1000, 300)
# print(sampleRNN)

## **Exercise 3.2** Decoding behavior from sequential neural data using RNN

Seq2seq

### **Step 1:** Download the dataset. 

It is a classic monkey reach dataset with obstacles (MC_Maze) fro Churchland et al. ([more info](https://dandiarchive.org/dandiset/000140))

In [1]:
# ! pip install "dandi>=0.13.0"
# ! dandi download DANDI:000140/0.220113.0408
# ! pip install git+https://github.com/neurallatents/nlb_tools.git

In [10]:
from nlb_tools.nwb_interface import NWBDataset

# for simplicity, we are using NLB tools 
dataset = NWBDataset("/content/000140/sub-Jenkins", "*", 
                     split_heldout=True)

# to view the dataset, uncomment the next line
# dataset.data

### **Step 2**: Select the fields that we are going to use

The continuous data provided with the MC_Maze datasets includes:

* `cursor_pos` - x and y position of the cursor controlled by the monkey
* `eye_pos` - x and y position of the monkey's point of gaze on the screen, in mm
* `hand_pos` - x and y position of the monkey's hand, in mm
* `hand_vel` - x and y velocities of the monkey's hand, in mm/s, computed offline using np.gradient
* `spikes` - spike times binned at 1 ms

Here we will try picking a single aspect of behavior (e.g. `hand_vel`) and decoding it from the spike data `spikes` using an RNN decoder.


### **Step 3:** Trialize and visualize the data
Surprisingly autonomous -> movement onset

Help on class NWBDataset in module nlb_tools.nwb_interface:

class NWBDataset(builtins.object)
 |  NWBDataset(fpath, prefix='', split_heldout=True, skip_fields=[])
 |  
 |  A class for loading/preprocessing data from NWB files for
 |  the NLB competition
 |  
 |  Methods defined here:
 |  
 |  __init__(self, fpath, prefix='', split_heldout=True, skip_fields=[])
 |      Initializes an NWBDataset, loading data from 
 |      the indicated file(s)
 |      
 |      Parameters
 |      ----------
 |      fpath : str
 |          Either the path to an NWB file or to a directory
 |          containing NWB files
 |      prefix : str, optional
 |          A pattern used to filter the NWB files in directory
 |          by name. By default, prefix='' loads all .nwb files in
 |          the directory. Please refer to documentation for
 |          the `glob` module for more details: 
 |          https://docs.python.org/3/library/glob.html
 |      split_heldout : bool, optional
 |          Whether to l

## **Exercise 3.3** Auto-encoding sequential neural data with RNN-based LFADS model 

## **Exercise 3.4\*:** Non-autonomous LFADS: identifiability of control inputs



# References

1. Izhikevich, Eugene M. Dynamical systems in neuroscience. MIT press, 2007.